In [1]:
import pandas as pd

In [5]:
df_17 = pd.read_csv('/Users/xuanedx1/github/outage-data-scraper/data/bluefire/POUS_Export_CityByUtility_Raw_2017.csv', encoding='utf-16')
df_18 = pd.read_csv('/Users/xuanedx1/github/outage-data-scraper/data/bluefire/POUS_Export_CityByUtility_Raw_2018.csv', encoding='utf-16')
df_19 = pd.read_csv('/Users/xuanedx1/github/outage-data-scraper/data/bluefire/POUS_Export_CityByUtility_Raw_2019.csv', encoding='utf-16')
df_20 = pd.read_csv('/Users/xuanedx1/github/outage-data-scraper/data/bluefire/POUS_Export_CityByUtility_Raw_2020.csv', encoding='utf-16')
df_21 = pd.read_csv('/Users/xuanedx1/github/outage-data-scraper/data/bluefire/POUS_Export_CityByUtility_Raw_2021.csv', encoding='utf-16')
df_22 = pd.read_csv('/Users/xuanedx1/github/outage-data-scraper/data/bluefire/POUS_Export_CityByUtility_Raw_2022.csv', encoding='utf-16')
df_23 = pd.read_csv('/Users/xuanedx1/github/outage-data-scraper/data/bluefire/POUS_Export_CityByUtility_Raw_2023.csv', encoding='utf-16')


In [9]:
df_22.head(50)

,UtilityName,StateName,CountyName,CityName,CountyFIPS,CustomersTracked,CustomersOut,RecordDateTime
0,4 Rivers Electric Coop,Kansas,Anderson,Unknown,20003.0,973,1,2022-01-01 23:58:04
1,4 Rivers Electric Coop,Kansas,Anderson,Unknown,20003.0,973,0,2022-01-02 05:57:59
2,4 Rivers Electric Coop,Kansas,Anderson,Unknown,20003.0,973,1,2022-01-02 07:56:10
3,4 Rivers Electric Coop,Kansas,Anderson,Unknown,20003.0,973,0,2022-01-02 08:56:17
4,4 Rivers Electric Coop,Kansas,Anderson,Unknown,20003.0,973,1,2022-01-02 11:56:07
5,4 Rivers Electric Coop,Kansas,Anderson,Unknown,20003.0,973,0,2022-01-02 12:56:50
6,4 Rivers Electric Coop,Kansas,Anderson,Unknown,20003.0,973,1,2022-01-02 13:56:54
7,4 Rivers Electric Coop,Kansas,Anderson,Unknown,20003.0,973,0,2022-01-02 18:57:10
8,4 Rivers Electric Coop,Kansas,Anderson,Unknown,20003.0,973,1,2022-01-02 19:57:48
9,4 Rivers Electric Coop,Kansas,Anderson,Unknown,20003.0,123,0,2022-01-02 20:57:09
